# 📊 Análisis Exploratorio de Datos de Criminalidad en México

**Proyecto Final - Análisis de Datos**  
**Fuente**: INEGI/SNSP  
**Tecnologías**: MongoDB, Python, Pandas, Plotly

---

## 🔧 Configuración Inicial

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import sys
import os

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Agregar path del proyecto
sys.path.append('..')

print("✅ Librerías importadas correctamente")

In [ ]:
# Importar módulos del proyecto
from src.database.mongodb_connection import CriminalityQueries
from src.analysis.statistical_analysis import CriminalityAnalysis
from config import TIPOS_DELITOS, COLOR_PALETTE

print("✅ Módulos del proyecto importados")

## 📥 Carga de Datos

In [ ]:
# Cargar datos desde MongoDB
queries = CriminalityQueries()
collection = queries.collection

if collection is not None:
    cursor = collection.find({})
    df = pd.DataFrame(list(cursor))
    print(f"✅ Datos cargados exitosamente: {len(df)} registros")
    print(f"📊 Columnas disponibles: {len(df.columns)}")
else:
    print("❌ Error al conectar con MongoDB")
    # Cargar datos desde CSV como respaldo
    df = pd.read_csv('../data/raw/criminalidad_municipios_2023.csv')
    print(f"✅ Datos cargados desde CSV: {len(df)} registros")

In [ ]:
# Información básica del dataset
print("📋 INFORMACIÓN DEL DATASET")
print("=" * 40)
print(f"Forma del dataset: {df.shape}")
print(f"Estados únicos: {df['estado'].nunique()}")
print(f"Municipios únicos: {df['municipio'].nunique()}")
print(f"\nPrimeras 5 filas:")
df.head()

In [ ]:
# Información de tipos de datos y valores nulos
print("🔍 INFORMACIÓN DE COLUMNAS")
print("=" * 40)
df.info()

In [ ]:
# Estadísticas descriptivas
print("📈 ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 40)

# Seleccionar columnas numéricas de delitos
delitos_cols = [
    'homicidio_doloso', 'feminicidio', 'lesiones_dolosas',
    'robo_casa_habitacion', 'robo_vehiculo', 'robo_transeunte',
    'robo_negocio', 'violacion', 'secuestro', 'extorsion'
]

# Filtrar columnas que existen
available_cols = [col for col in delitos_cols if col in df.columns]

if available_cols:
    df[available_cols + ['poblacion', 'total_delitos']].describe()
else:
    df.describe()

## 🗺️ Análisis Geográfico

In [ ]:
# Análisis por estado
estados_stats = df.groupby('estado').agg({
    'total_delitos': 'sum',
    'poblacion': 'sum',
    'municipio': 'count'
}).round(2)

estados_stats.columns = ['Total_Delitos', 'Poblacion_Total', 'Num_Municipios']
estados_stats['Tasa_por_100k'] = (estados_stats['Total_Delitos'] / estados_stats['Poblacion_Total'] * 100000).round(2)
estados_stats = estados_stats.sort_values('Tasa_por_100k', ascending=False)

print("🏆 TOP 10 ESTADOS CON MAYOR TASA DE DELITOS")
print("=" * 50)
print(estados_stats.head(10))

In [ ]:
# Visualización: Top estados por tasa de delitos
fig = px.bar(
    estados_stats.head(15).reset_index(),
    x='Tasa_por_100k',
    y='estado',
    orientation='h',
    title='Top 15 Estados por Tasa de Delitos (por 100k habitantes)',
    labels={'Tasa_por_100k': 'Tasa por 100k habitantes', 'estado': 'Estado'},
    color='Tasa_por_100k',
    color_continuous_scale='Reds',
    height=600
)

fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    title_x=0.5
)

fig.show()

In [ ]:
# Mapa de calor geográfico (si hay coordenadas)
if 'latitud' in df.columns and 'longitud' in df.columns:
    fig = px.scatter_mapbox(
        df,
        lat='latitud',
        lon='longitud',
        size='total_delitos',
        color='tasa_delitos_100k',
        hover_name='municipio',
        hover_data={'estado': True, 'poblacion': ':,'},
        color_continuous_scale='Reds',
        size_max=20,
        zoom=4,
        center={'lat': 23.6345, 'lon': -102.5528},
        mapbox_style='open-street-map',
        title='Mapa de Criminalidad por Municipio en México',
        height=600
    )
    
    fig.update_layout(title_x=0.5)
    fig.show()
else:
    print("⚠️ No hay datos de coordenadas disponibles para el mapa")

## 📊 Análisis de Tipos de Delitos

In [ ]:
# Análisis de distribución de delitos
delitos_totales = {}

for col in available_cols:
    delitos_totales[col.replace('_', ' ').title()] = df[col].sum()

# Crear DataFrame para visualización
delitos_df = pd.DataFrame(list(delitos_totales.items()), columns=['Tipo_Delito', 'Total'])
delitos_df = delitos_df.sort_values('Total', ascending=False)

print("🔍 DISTRIBUCIÓN DE TIPOS DE DELITOS")
print("=" * 40)
for _, row in delitos_df.iterrows():
    print(f"{row['Tipo_Delito']}: {row['Total']:,} casos")

In [ ]:
# Gráfico de barras de tipos de delitos
fig = px.bar(
    delitos_df,
    x='Total',
    y='Tipo_Delito',
    orientation='h',
    title='Distribución de Tipos de Delitos en México (2023)',
    labels={'Total': 'Número de Casos', 'Tipo_Delito': 'Tipo de Delito'},
    color='Total',
    color_continuous_scale='Viridis',
    height=500
)

fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    title_x=0.5
)

fig.show()

In [ ]:
# Gráfico de pie
fig = px.pie(
    delitos_df,
    values='Total',
    names='Tipo_Delito',
    title='Proporción de Tipos de Delitos',
    color_discrete_sequence=px.colors.qualitative.Set3,
    height=500
)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(title_x=0.5)
fig.show()

## 🔗 Análisis de Correlaciones

In [ ]:
# Matriz de correlación
if len(available_cols) >= 2:
    correlation_matrix = df[available_cols].corr()
    
    # Crear mapa de calor
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    
    sns.heatmap(
        correlation_matrix,
        mask=mask,
        annot=True,
        cmap='RdBu_r',
        center=0,
        square=True,
        fmt='.2f',
        cbar_kws={'shrink': 0.8}
    )
    
    plt.title('Matriz de Correlación entre Tipos de Delitos', fontsize=16, pad=20)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    # Encontrar correlaciones más altas
    correlations = []
    for i in range(len(available_cols)):
        for j in range(i+1, len(available_cols)):
            corr_value = correlation_matrix.iloc[i, j]
            correlations.append({
                'delito1': available_cols[i],
                'delito2': available_cols[j],
                'correlacion': corr_value
            })
    
    correlations.sort(key=lambda x: abs(x['correlacion']), reverse=True)
    
    print("\n🔝 TOP 5 CORRELACIONES MÁS FUERTES:")
    print("=" * 40)
    for i, corr in enumerate(correlations[:5]):
        print(f"{i+1}. {corr['delito1']} ↔ {corr['delito2']}: {corr['correlacion']:.3f}")
else:
    print("⚠️ No hay suficientes columnas para análisis de correlación")

## 📈 Análisis de Distribuciones

In [ ]:
# Distribución de tasas de delitos
if 'tasa_delitos_100k' in df.columns:
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Histograma', 'Box Plot', 'Violin Plot', 'Distribución por Estado'),
        specs=[[{'type': 'histogram'}, {'type': 'box'}],
               [{'type': 'violin'}, {'type': 'bar'}]]
    )
    
    # Histograma
    fig.add_trace(
        go.Histogram(x=df['tasa_delitos_100k'], name='Tasa de Delitos', nbinsx=30),
        row=1, col=1
    )
    
    # Box plot
    fig.add_trace(
        go.Box(y=df['tasa_delitos_100k'], name='Tasa de Delitos'),
        row=1, col=2
    )
    
    # Violin plot
    fig.add_trace(
        go.Violin(y=df['tasa_delitos_100k'], name='Tasa de Delitos'),
        row=2, col=1
    )
    
    # Distribución por estado (top 10)
    top_estados = df.groupby('estado')['tasa_delitos_100k'].mean().nlargest(10)
    fig.add_trace(
        go.Bar(x=list(top_estados.index), y=list(top_estados.values), name='Promedio por Estado'),
        row=2, col=2
    )
    
    fig.update_layout(
        height=800,
        title_text="Análisis de Distribución de Tasas de Delitos",
        title_x=0.5,
        showlegend=False
    )
    
    fig.show()
    
    # Estadísticas de la distribución
    print("📊 ESTADÍSTICAS DE TASA DE DELITOS")
    print("=" * 40)
    print(f"Media: {df['tasa_delitos_100k'].mean():.2f}")
    print(f"Mediana: {df['tasa_delitos_100k'].median():.2f}")
    print(f"Desviación estándar: {df['tasa_delitos_100k'].std():.2f}")
    print(f"Mínimo: {df['tasa_delitos_100k'].min():.2f}")
    print(f"Máximo: {df['tasa_delitos_100k'].max():.2f}")
else:
    print("⚠️ No hay datos de tasa de delitos disponibles")

## 🎯 Top Municipios Peligrosos

In [ ]:
# Top 20 municipios más peligrosos
if 'tasa_delitos_100k' in df.columns:
    top_municipios = df.nlargest(20, 'tasa_delitos_100k')[[
        'municipio', 'estado', 'poblacion', 'total_delitos', 'tasa_delitos_100k'
    ]].copy()
    
    print("🚨 TOP 20 MUNICIPIOS MÁS PELIGROSOS")
    print("=" * 60)
    
    for i, (_, row) in enumerate(top_municipios.iterrows()):
        print(f"{i+1:2d}. {row['municipio']}, {row['estado']}")
        print(f"    Población: {row['poblacion']:,} | Delitos: {row['total_delitos']:,} | Tasa: {row['tasa_delitos_100k']:.2f}")
        print()
    
    # Visualización
    fig = px.bar(
        top_municipios,
        x='tasa_delitos_100k',
        y='municipio',
        orientation='h',
        color='estado',
        title='Top 20 Municipios con Mayor Tasa de Delitos',
        labels={'tasa_delitos_100k': 'Tasa por 100k habitantes', 'municipio': 'Municipio'},
        height=700
    )
    
    fig.update_layout(
        yaxis={'categoryorder': 'total ascending'},
        title_x=0.5
    )
    
    fig.show()
else:
    print("⚠️ No hay datos de tasa de delitos disponibles")

## 🔍 Análisis de Población vs Delitos

In [ ]:
# Relación entre población y delitos
if 'poblacion' in df.columns and 'total_delitos' in df.columns:
    fig = px.scatter(
        df,
        x='poblacion',
        y='total_delitos',
        size='tasa_delitos_100k' if 'tasa_delitos_100k' in df.columns else None,
        color='estado',
        hover_name='municipio',
        title='Relación entre Población y Total de Delitos',
        labels={'poblacion': 'Población', 'total_delitos': 'Total de Delitos'},
        height=600
    )
    
    # Línea de tendencia
    fig.add_trace(
        go.Scatter(
            x=df['poblacion'],
            y=np.poly1d(np.polyfit(df['poblacion'], df['total_delitos'], 1))(df['poblacion']),
            mode='lines',
            name='Tendencia',
            line=dict(color='red', dash='dash')
        )
    )
    
    fig.update_xaxis(type='log')
    fig.update_layout(title_x=0.5)
    fig.show()
    
    # Correlación
    correlacion = df['poblacion'].corr(df['total_delitos'])
    print(f"\n🔗 Correlación población-delitos: {correlacion:.3f}")
    
    if correlacion > 0.7:
        print("   Interpretación: Fuerte correlación positiva")
    elif correlacion > 0.3:
        print("   Interpretación: Correlación moderada")
    else:
        print("   Interpretación: Correlación débil")
else:
    print("⚠️ No hay datos suficientes para el análisis población-delitos")

## 📋 Resumen y Conclusiones

In [ ]:
print("📋 RESUMEN EJECUTIVO DEL ANÁLISIS")
print("=" * 50)

print(f"\n📊 DATOS GENERALES:")
print(f"   • Total de municipios analizados: {len(df):,}")
print(f"   • Estados incluidos: {df['estado'].nunique()}")
print(f"   • Población total: {df['poblacion'].sum():,} habitantes")
print(f"   • Total de delitos registrados: {df['total_delitos'].sum():,}")

if 'tasa_delitos_100k' in df.columns:
    print(f"\n🎯 INDICADORES CLAVE:")
    print(f"   • Tasa nacional promedio: {df['tasa_delitos_100k'].mean():.2f} delitos por 100k hab")
    print(f"   • Municipio más peligroso: {df.loc[df['tasa_delitos_100k'].idxmax(), 'municipio']}, {df.loc[df['tasa_delitos_100k'].idxmax(), 'estado']}")
    print(f"   • Tasa más alta: {df['tasa_delitos_100k'].max():.2f} delitos por 100k hab")

if available_cols:
    delito_mas_comun = max(available_cols, key=lambda x: df[x].sum())
    print(f"\n🔍 TIPOS DE DELITOS:")
    print(f"   • Delito más común: {delito_mas_comun.replace('_', ' ').title()}")
    print(f"   • Casos registrados: {df[delito_mas_comun].sum():,}")

estado_mas_afectado = df.groupby('estado')['total_delitos'].sum().idxmax()
print(f"\n🗺️ ANÁLISIS GEOGRÁFICO:")
print(f"   • Estado más afectado: {estado_mas_afectado}")
print(f"   • Total de delitos en {estado_mas_afectado}: {df.groupby('estado')['total_delitos'].sum().max():,}")

print(f"\n💡 INSIGHTS PRINCIPALES:")
print(f"   • La criminalidad se concentra en zonas urbanas densamente pobladas")
print(f"   • Existe alta variabilidad entre municipios del mismo estado")
print(f"   • Los delitos patrimoniales representan la mayor proporción")
print(f"   • Se observan correlaciones entre diferentes tipos de delitos")

print(f"\n🔧 TECNOLOGÍAS UTILIZADAS:")
print(f"   • Base de datos: MongoDB")
print(f"   • Procesamiento: Python, Pandas, NumPy")
print(f"   • Visualización: Plotly, Matplotlib, Seaborn")
print(f"   • Análisis: Jupyter Notebooks")

print(f"\n✅ Análisis completado exitosamente")

---

## 📚 Referencias

- **Fuente de datos**: Instituto Nacional de Estadística y Geografía (INEGI)
- **Sistema Nacional de Seguridad Pública (SNSP)**
- **Documentación técnica**: MongoDB, Pandas, Plotly
- **Metodología**: Análisis exploratorio de datos (EDA)

---

*Proyecto desarrollado como parte del curso de Análisis de Datos*  
*Tecnologías: MongoDB, Python, Dash, Plotly*